# PYTHON DB INTERACTION

In [28]:
import os
import pymongo
from pymongo import MongoClient
from glob import glob
from util import database, create_file_obj

clients_dir = "/Volumes/GoogleDrive/Shared drives/Buffalo Graphics/clients"
db = database()



In [32]:
import re
from pprint import pprint
import json

import re
from pprint import pprint
import json


def clients_from_dir(dir):
    _clients = db["clients"]
    inserted = []

    for client in os.listdir(dir):

        if _clients.find_one({"name": client}) is None:

            client_obj = {
                "name": client,
                "file_path": os.path.join(clients_dir, client)
            }

            client_id = _clients.insert_one(client_obj).inserted_id

            if client_id is not None:
                inserted.append(
                    ({"_id": client_id, "name": client, }))

    pprint({
        "created": f"{len(inserted)} new clients",
        "client_count": _clients.count_documents({})
    })


clients_from_dir(clients_dir)



def clients_from_dir(dir: str) -> None:
    _clients = db["clients"]
    clients = []
    for c in _clients.find({}):
        clients.append(c.get("name"))
    inserted = []

    for client in os.listdir(dir):
        
        if client in clients:

            client_obj = {
                "name": client,
                "file_path": os.path.join(clients_dir, client)
            }

            client_id = _clients.insert_one(client_obj).inserted_id
            print(client_id)
            if client_id is not None:
                inserted.append(
                    ({"_id": client_id, "name": client, }))

    # pprint({
    #     "created": f"{len(inserted)} new clients",
    #     "client_count": _clients.count_documents({})
    # })


clients_from_dir(clients_dir)


{'client_count': 22, 'created': '22 new clients'}


In [33]:
def format_job_name(job_full_path, dir_name):
    name = job_full_path.replace(dir_name+'/', '')

    if name == "PRINT":
        name = "default"

    name = name.replace("/PRINT", '')
    
    return name

In [73]:
def job_file_doc(job):
    print(job)
    _file = db["files"]
    file_names = glob(f"{job['dir']}/*.pdf")
    files = []
    for file in file_names: 
        files.append(create_file_obj(job["client_id"], job["_id"],file))
            
    if len(files) != 0:
        return _file.insert_many(files).inserted_ids

In [42]:
def create_job_doc(obj, job_full_path):
    # obj = { "name": "Company Name" }
    dir = os.path.join(clients_dir, obj['name'])


    job = {
        "client_id": obj["_id"],
        "name": format_job_name(job_full_path, dir),
        "dir": job_full_path,
    }

    
    return job


In [67]:
def link_jobs(q):
    _clients = db["clients"]
    _jobs = db["jobs"]

    client = _clients.find_one(q)
    dir_name = os.path.join(clients_dir, client['name'])

    jobs = []

    for job_full_path in glob(f"{dir_name}/**/**/PRINT", recursive=True):
        job = create_job_doc(client, job_full_path)
        jobs.append(_jobs.insert_one(job).inserted_id)

    return _clients.update_one(q, {"$set": {'jobs': jobs}}).upserted_id


In [75]:
clients = []

for client in db["clients"].find():
    print(client)
    for job in db["jobs"].find({"client_id":client["_id"]}):
        if job.get("client_id",None)!=None:
            job_files = job_file_doc(job)
            db["jobs"].update_one(job,{"$set":{'files':job_files}})
    # if 'name' in c:
    #     print(link_jobs({'name': c['name']}))


{'_id': ObjectId('60a568894d5d31558e12739f'), 'name': 'Bell Concrete', 'file_path': '/Volumes/GoogleDrive/Shared drives/Buffalo Graphics/clients/Bell Concrete', 'jobs': [ObjectId('60a576ae4d5d31558e127409'), ObjectId('60a576ae4d5d31558e12740a')]}
{'_id': ObjectId('60a568c74d5d31558e1273b5'), 'client_id': ObjectId('60a568894d5d31558e12739f'), 'name': 'mixer', 'dir': '/Volumes/GoogleDrive/Shared drives/Buffalo Graphics/clients/Bell Concrete/mixer/PRINT'}
bell_cc_USDOT_qty_2.pdf
2
bell_cc_MS_number_qty_2.pdf
2
bell_cc_drum_graphic_qty_2.pdf
2
bell_cc_door_graphic_qty_2.pdf
2
{'_id': ObjectId('60a568c74d5d31558e1273b6'), 'client_id': ObjectId('60a568894d5d31558e12739f'), 'name': 'mixer', 'dir': '/Volumes/GoogleDrive/Shared drives/Buffalo Graphics/clients/Bell Concrete/mixer/PRINT'}
bell_cc_USDOT_qty_2.pdf
2
bell_cc_MS_number_qty_2.pdf
2
bell_cc_drum_graphic_qty_2.pdf
2
bell_cc_door_graphic_qty_2.pdf
2
{'_id': ObjectId('60a56e174d5d31558e1273d9'), 'client_id': ObjectId('60a568894d5d31558e12

## DELETE DB

In [16]:
db["clients"].delete_many({})
db["jobs"].delete_many({})
db["files"].delete_many({})